In [2]:
from smf.db_engine import DbEngine
import pandas as pd
import numpy as np
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices
import eikon as ek
ek.set_app_key('403255e90c7647afafbfb5c0000d60ac4c8cc536')

In [3]:
db = DbEngine()

-I- Successful database connection


In [4]:
close_prices = db.fetch_data_series("close_price")
close_prices

""


In [5]:
final_companies1 = ["RDN.N","HIG.N","PHM.N","HMST.OQ","BHE.N","JPM.N","MAS.N","FITB.OQ","MS.N","DHI.N","FFWM.OQ","ABBV.N", "BMY.N"]
final_companies = [x.replace(".", "_") for x in final_companies1]
final_companies

['RDN_N',
 'HIG_N',
 'PHM_N',
 'HMST_OQ',
 'BHE_N',
 'JPM_N',
 'MAS_N',
 'FITB_OQ',
 'MS_N',
 'DHI_N',
 'FFWM_OQ',
 'ABBV_N',
 'BMY_N']

In [6]:
close_prices_12m = close_prices.loc["2018-11-18":, final_companies].fillna(method="ffill")

KeyError: "None of [Index(['RDN_N', 'HIG_N', 'PHM_N', 'HMST_OQ', 'BHE_N', 'JPM_N', 'MAS_N',\n       'FITB_OQ', 'MS_N', 'DHI_N', 'FFWM_OQ', 'ABBV_N', 'BMY_N'],\n      dtype='object')] are in the [columns]"

In [62]:
min_weight, max_weight = 0.05, 0.10
print("--------------------------------------")
print(f"Weight range: [{min_weight}, {max_weight}]")
print()
mu = expected_returns.mean_historical_return(close_prices_12m)
S = risk_models.CovarianceShrinkage(close_prices_12m).ledoit_wolf()

ef = EfficientFrontier(expected_returns=mu, cov_matrix=S, weight_bounds=(min_weight, max_weight))
raw_weights = ef.max_sharpe()
cleaned_weights = ef.clean_weights()

ef.portfolio_performance(verbose=True)
print(pd.Series(cleaned_weights).sort_values(ascending=False))

--------------------------------------
Weight range: [0.05, 0.1]

Expected annual return: 30.5%
Annual volatility: 16.2%
Sharpe Ratio: 1.75
DHI_N      0.10000
MAS_N      0.10000
BHE_N      0.10000
PHM_N      0.10000
HIG_N      0.10000
RDN_N      0.10000
HMST_OQ    0.09441
JPM_N      0.05559
BMY_N      0.05000
ABBV_N     0.05000
FFWM_OQ    0.05000
MS_N       0.05000
FITB_OQ    0.05000
dtype: float64


# Get current prices

In [53]:
curr_prices = ek.get_data(final_companies1, "CF_LAST")[0]

In [55]:
curr_prices['Instrument'] = curr_prices['Instrument'].apply(lambda x: x.replace(".", "_"))
curr_prices = curr_prices.set_index("Instrument")['CF_LAST']

In [63]:
da = DiscreteAllocation(cleaned_weights, curr_prices, total_portfolio_value=60000)
allocation, leftover = da.lp_portfolio()
print("Left Over: {}".format(leftover))

0 out of 13 tickers were removed
Left Over: 11.57


# Final Allocation

In [64]:
df = pd.DataFrame()
df['Number of shares'] = pd.Series(allocation)
df['Weights'] = pd.Series(cleaned_weights).sort_values(ascending=False)
df['Current Price'] = pd.Series(curr_prices)
df

,Number of shares,Weights,Current Price
RDN_N,238,0.10000,25.19
HIG_N,99,0.10000,60.30
PHM_N,152,0.10000,39.37
HMST_OQ,177,0.09441,32.06
BHE_N,176,0.10000,34.06
JPM_N,26,0.05559,129.78
MAS_N,133,0.10000,45.10
FITB_OQ,101,0.05000,29.49
MS_N,62,0.05000,48.11
DHI_N,110,0.10000,54.76


In [61]:
(df['Number of shares'] * df['Current Price']).sum()

59988.43

In [65]:
df.to_csv("optimisation1.csv")